In [ ]:
%%capture
!pip install wandb -qqq

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from google.colab import drive
import numpy as np
import cv2
import keras.backend as K

MODELS_PATH = "/content/gdrive/My Drive/magshimim/camerona/models/"
SSD_BASE = "ssd_base_model.h5"
SSD_BEST = "ssd_best_model.h5"
DATASET_PATH_DRIVE = "/content/gdrive/My Drive/magshimim/camerona/dataset/SSD/"
DATASET_PATH_COLAB = "/content/SSD/"
DATASET_IMG = DATASET_PATH_COLAB + "images/"
VAL_TEXT = DATASET_PATH_COLAB+"val.txt"
TRAIN_TEXT = DATASET_PATH_COLAB+"train.txt"
TEST_TEXT = DATASET_PATH_COLAB+"test.txt"

drive.mount('/content/gdrive/')

In [ ]:
!ln -s "/content/gdrive/My Drive/magshimim/camerona/dataset/SSD/" "/content/"

In [ ]:
import wandb
from wandb.keras import WandbCallback
!wandb login

In [ ]:
#input shape: (batch_size,20,2,2)
def computeArea(a):
  x1 = a[:,:,0,0]
  x2 = a[:,:,1,0]
  y1 = a[:,:,0,1]
  y2 = a[:,:,1,1]
  x_len = tf.math.subtract(x2,x1)
  y_len = tf.math.subtract(y2,y1)
  area = tf.math.multiply(x_len, y_len)
  return area

#input shape: (batch_size,20,2,2), (batch_size,20,2,2)
def IoU(pred_bbox, ground_truth):
  area_pred = computeArea(pred_bbox)
  area_gt = computeArea(ground_truth)

  inter_x1 = tf.math.maximum(pred_bbox[:,:,0,0], ground_truth[:,:,0,0])
  inter_x2 = tf.math.minimum(pred_bbox[:,:,1,0], ground_truth[:,:,1,0])
  inter_y1 = tf.math.maximum(pred_bbox[:,:,0,1], ground_truth[:,:,0,1])
  inter_y2 = tf.math.minimum(pred_bbox[:,:,1,1], ground_truth[:,:,1,1])

  y = tf.math.subtract(inter_y2,inter_y1)
  x = tf.math.subtract(inter_x2,inter_x1)

  area_inter = tf.math.multiply(x,y)

  area_union = tf.math.add(area_pred,area_gt)
  area_union = tf.math.subtract(area_union,area_inter)

  x_tf = tf.where(x<=0.0, True, False)
  y_tf = tf.where(y<=0.0, True, False)
  is_inter = tf.math.logical_or(x_tf,y_tf)
  acc = tf.where(is_inter, 0.0, tf.divide(area_inter, area_union))

  area_gt_tf = tf.where(area_gt==0.0, True, False)
  area_pred_tf = tf.where(area_pred==0.0, True, False)
  is_one_area = tf.math.logical_and(area_gt_tf,area_pred_tf)
  acc = tf.where(is_one_area, 1.0, acc)

  return acc

def boxLoss(pred_bbox, ground_truth):
  loss = tf.math.subtract(pred_bbox, ground_truth)
  loss = tf.math.abs(loss)
  loss = tf.math.sqrt(tf.math.sqrt(loss))
  loss = tf.math.multiply(10.0,loss)
  return loss

def boxLoss1(pred_bbox, ground_truth):
  acc = tf.math.pow(IoU(pred_bbox, ground_truth), 2)
  return tf.subtract(1.0, acc)*10

In [ ]:
def loadVGG16():
  model = keras.applications.VGG16(
    include_top = False,
    weights ="imagenet",
    input_tensor = None,
    input_shape = (256,256,3),
    pooling=None)
  return model

def buildModel():
  vgg16_backbone = loadVGG16()

	#making the backbone untrainable
  for layer in vgg16_backbone.layers:
    layer.trainable = False

  ####SSD layers
  ssd = keras.layers.Conv2D(1024, 3, dilation_rate=6, activation='relu', padding='same', name = "ssd_block1_conv1")(vgg16_backbone.output)

  ssd = keras.layers.Conv2D(1024, 1, activation='relu', padding='same', name = "ssd_block2_conv1")(ssd)

  ssd = keras.layers.Conv2D(256, 1, activation='relu', padding='same', name = "ssd_block3_conv1")(ssd)
  ssd = keras.layers.Conv2D(512, 3, strides=2, activation='relu', padding='same', name = "ssd_block3_conv2")(ssd)

  ssd = keras.layers.Conv2D(128, 1, activation='relu', padding='same', name = "ssd_block4_conv1")(ssd)
  ssd = keras.layers.Conv2D(256, 3, strides=2, activation='relu', padding='same', name = "ssd_block4_conv2")(ssd)

  ssd = keras.layers.Conv2D(128, 1, activation='relu', padding='same', name = "ssd_block5_conv1")(ssd)
  ssd = keras.layers.Conv2D(256, 3, activation='relu', padding='same', name = "ssd_block5_conv2")(ssd)

  ssd = keras.layers.Conv2D(128, 1, activation='relu', padding='same', name = "ssd_block6_conv1")(ssd)
  ssd = keras.layers.Conv2D(256, 3, activation='relu', padding='same', name = "ssd_block6_conv2")(ssd)
  ####

	####output layers
  #class layers
  flt = keras.layers.Flatten()(ssd)
  class_output = keras.layers.Dense(20, activation='sigmoid', name='output_class')(flt)
  
  #location layers
  ssd = keras.layers.Conv2D(20, 3, activation='relu', padding='same', name = "output_loc_b4_reshape")(ssd)
  loc_output = keras.layers.Reshape((20,2,2), name='output_loc')(ssd)
  ####

  #model and optimizer
  model = keras.Model(inputs=vgg16_backbone.input, outputs=[class_output, loc_output])
  opt = keras.optimizers.Adam(learning_rate=0.01)

  #custom loss and metrics
  metrics = {'output_class':keras.metrics.BinaryAccuracy(), 'output_loc':IoU}

  losses = {'output_class':"binary_crossentropy", 'output_loc':boxLoss}

  model.compile(optimizer = opt, loss=losses, metrics=metrics)
  print(model.summary())

  return model

In [ ]:
def buildData(data, show_every=250):
  x = []
  y_guess = []
  y_loc = []
  print(len(data))
  cnt1 = 1
  for line in data:
    if cnt1%show_every == 0:
      print(cnt1)
    cnt1 += 1
    l = line.split(' ')
    try:
      #some data was deleted and finding it is a pain, this is easier
      img = cv2.imread(DATASET_IMG+l[0])
      img = cv2.resize(img, (256,256))/255.0
    except:
      continue
    y_class = np.zeros(20)
    y_location = np.zeros((20,2,2))#[[[x1, y1],[x2, y2]]...20]
    if len(l) > 1:
      l = l[1:]
      cnt = 0
      try:
        for points in l:
          points = points.split(',')
          y_location[cnt][0][0] = tf.cast(float(points[0]), tf.float32)
          y_location[cnt][0][1] = tf.cast(float(points[1]), tf.float32)
          y_location[cnt][1][0] = tf.cast(float(points[2]), tf.float32)
          y_location[cnt][1][1] = tf.cast(float(points[3]), tf.float32)
          y_class[cnt] = 1
          cnt += 1
      except:
        continue
    x.append(img)
    y_guess.append(y_class)
    y_loc.append(y_location)
  x, y_guess, y_loc = np.array(x), np.array(y_guess), np.array(y_loc)
  print(x.shape, y_guess.shape, y_loc.shape)
  return x, y_guess, y_loc

def dataFromFile(path):
	f = open(path, 'r')
	ret = f.read().split('\n')
	f.close()
	return ret

def splitArrayToArrays(arr, am):
  l = len(arr)
  to_ret = []
  i = 0
  cnt = 0
  while i < l:
    cnt += 1
    a = []
    while i < am*cnt and i < l:
      a.append(arr[i])
      i+=1
    to_ret.append(a)
  return to_ret

Building model and saving it

In [ ]:
model = buildModel()
model.save(MODELS_PATH+SSD_BASE)

Callbacks

In [ ]:
wandb.init(project="SSD - face detector")

best_callback = keras.callbacks.ModelCheckpoint(MODELS_PATH+SSD_BEST, save_best_only=True)
wandb_callback = WandbCallback()

**checking the default outputs**

In [ ]:
val_data = [dataFromFile(VAL_TEXT)[0]]
print(val_data)
val_x, val_y_guess, val_y_loc = buildData(val_data)
y1, y2 = model.predict(val_x)

print(y2)

In [ ]:
#getting validation data
val_data = dataFromFile(VAL_TEXT)

#getting test pre-data
train_data = splitArrayToArrays(dataFromFile(TRAIN_TEXT), 700)

In [ ]:
val_x, val_y_guess, val_y_loc = buildData(val_data, 100)

In [ ]:
cnt = 1
l = len(train_data)
for data in train_data:
  print("\n\niteration {0}/{1}".format(cnt, l))
  train_x, train_y_guess, train_y_loc = buildData(data, 100)

  history = model.fit(train_x,
          [train_y_guess, train_y_loc],
          epochs=16, 
          validation_data=(val_x, [val_y_guess, val_y_loc]),
          #callbacks=[best_callback,wandb_callback],
          batch_size=16)
  to_save = keras.models.load_model(MODELS_PATH+SSD_BEST, compile=False)
  to_save.save(MODELS_PATH+"ssd_" + str(cnt) + ".h5")
  cnt += 1

**checking outputs after trining**

In [ ]:
x, y1, y1 = buildData([val_data[0]], 100)
y1, y2 = model.predict(val_x)
print(y2)

In [ ]:
best_models = []
for i in range(15, 19):
  best_models.append(keras.models.load_model(MODELS_PATH+"ssd_" + str(i) + ".h5"))

In [ ]:
#loading test data
test_data = splitArrayToArrays(dataFromFile(TEST_TEXT), 1500)
test_x, test_y_guess, test_y_loc = buildData(test_data[0])

In [ ]:
#testing all the models
best_acc = 0
best_model = None
cnt = 15
print(best_models[0].metrics_names)
for model in best_models:
  print(cnt)
  print(model.evaluate(test_x, [test_y_guess, test_y_loc]),"\n")
  cnt += 1

In [ ]:
model = buildModel()

In [ ]:
data = dataFromFile(TRAIN_TEXT)[:100]
test_x, test_y_guess, test_y_loc = buildData(data, 10)

In [ ]:
model.fit(test_x, 
          [test_y_guess, test_y_loc],
          epochs=24)